### Importing Libraries

In [ ]:

import requests
import json
import pandas as pd
import time, datetime

In [30]:
class Collector:
    def __init__(self, start_date, end_date, coin):
        self.start_date = start_date
        self.end_date = end_date
        self.coin = coin

    def a(self):
        try:
            api_url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@moeda='{self.coin}'&@dataInicial='{self.start_date}'&@dataFinalCotacao='{self.end_date}'&$top=10000&$filter=tipoBoletim%20eq%20'Fechamento'&$format=json&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim"
            response = requests.get(api_url)
            response.raise_for_status()
            
            if response.status_code == 200:
                data = response.json()
                folder_filename = "C:/Tecnology/Projects/bacen-ptax-data-processing/data/raw/ptax_data"
                with open(folder_filename, "w") as file:
                    json.dump(data, file)
            else:
                print('Failed to get data from API.')
                
        except requests.exceptions.RequestException as e:
            print('Failed to get data from API. Error: ', e)


    
        
    